<a href="https://colab.research.google.com/github/antoniomenegon/US-ML_Lessons/blob/master/Code/Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Time Series forecasting

In this example notebook, we will try to predict a simple time series with 3 different ML models:


1.   Simple ANN
2.   LSTM
3.   CNN

Given this, the notebook consists in:


*   A first section, with all the setups and the relevant preprocessing
*   A section where all the different models are presented and implemented
*   A final section, with the comparison of all the different models analsysed







## Preliminaries

In this section, we will setup all the imports and utilities used later on.

### Setup

In [0]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import tensorflow as tf

keras = tf.keras
%load_ext tensorboard

### Utilities

In [0]:
class TimeSeries(object):

  def __init__(self, time, slope, baseline, amplitude, noise_level, period, phase=0):
    self.time = time
    self.slope = slope
    self.baseline = baseline
    self.amplitude = amplitude
    self.noise_level = noise_level
    self.period = period
    self.phase = phase
    self.seed = 42

  def get_time_series(self):
    return self.baseline + self._trend() + self._seasonality() + self._white_noise()

  def _trend(self):
    return self.slope * self.time

  @staticmethod
  def _seasonal_pattern(season_time):
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))
    
  def _seasonality(self):
    season_time = ((self.time + self.phase) % self.period) / self.period
    return self.amplitude * self._seasonal_pattern(season_time)
  
  def _white_noise(self):
    rnd = np.random.RandomState(self.seed)
    return rnd.randn(len(self.time)) * self.noise_level

In [0]:
class Normalizer:

  def __init__(self):
    self.mean = None
    self.std = None

  def fit(self, data):
    self.mean = data.mean()
    self.std = data.std()
  
  def transform(self, data):
    return (data - self.mean)/self.std

  def inverse_transform(self, data):
    return data*self.std + self.mean

In [0]:
def plot_series(time, series, prediction=None, label=None,
                format="-", start=0, end=None):
    if prediction is None:
      plt.plot(time[start:end], series[start:end], format)
    else:
      plt.plot(time[start:end], series[start:end], format, label=label[0])
      plt.plot(time[start:end], prediction[start:end], format, label=label[1])
      plt.legend(fontsize=14)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
  

def sequential_window_dataset(series, window_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1))
    ds = ds.map(lambda window: (window[:-1], window[1:]))
    return ds.batch(1).prefetch(1)


def data_split(data, time, split_portion):
  n = len(data)
  split_time = int(split_portion*n)
  time_train = time[:split_time]
  x_train = series[:split_time]
  time_valid = time[split_time:]
  x_valid = series[split_time:]
  return x_train, time_train, x_valid, time_valid

In [0]:
def tb_log_path(name):
  return os.path.join("logs", name)

### Data

In [0]:
time = np.arange(4 * 365 + 1)
slope = 0.05
baseline = 10
amplitude = 40
period = 365
noise_level = 5
seed=42

series = TimeSeries(time,
                    slope,
                    baseline,
                    amplitude,
                    noise_level,
                    period).get_time_series()

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

In [0]:
SPLIT = .70
x_train, time_train, x_valid, time_valid = data_split(series, time, SPLIT)

In [0]:
WINDOW_SIZE = 30
train_set = sequential_window_dataset(x_train, WINDOW_SIZE)
valid_set = sequential_window_dataset(x_valid, WINDOW_SIZE)

## Models

In this section, the different models are implemented and investigated.

Before doing so, we introduce some settings used later on.

### Hyperparameters and Configurations

Objects that collect the hyperparameters and the main configurations used in the training phase.

In [0]:
class HParams(object):

  def __init__(self):
    self.LAYERS_DIM = []
    self.FILTERS = []
    self.KERNEL_SIZE = []
    self.STRIDES = None


class TrainConfigs(object):

  def __init__(self):
    self.EPOCHS = 100
    self.LOSS = None
    self.CALLBACKS = []
    self.METRICS = []
    self.OPTIMIZER = None

Callbacks used during training

In [0]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

def model_checkpoint(name):
  return keras.callbacks.ModelCheckpoint(name+".h5", save_best_only=True)
    
def early_stopping(patience=50):
  return keras.callbacks.EarlyStopping(patience=patience)

def lr_schedule():
  return keras.callbacks.LearningRateScheduler(
            lambda epoch: 1e-8 * 10**(epoch / 20))

def tensorboard_callback(name):
  logdir = os.path.join("logs", name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

Dictionary to store all the relevant comparisons

In [0]:
mae = {}

### Simple ANN

#### Model definition

In [0]:
def build_model(hparams, log_summary=True):
  
  keras.backend.clear_session()
  tf.random.set_seed(42)
  np.random.seed(42)

  num_layers = len(hparams.LAYERS_DIM)

  model = keras.models.Sequential()
  model.add(
      keras.layers.Dense(hparams.LAYERS_DIM[0],
                        batch_input_shape=[1, None, 1])
  )
  if num_layers > 1:
    for dim in hparams.LAYERS_DIM[1:]:
      model.add(keras.layers.Dense(dim))
  model.add(keras.layers.Lambda(lambda x: x * 200.0))

  if log_summary:
    print("\n", model.summary(), "\n")

  return model

#### Training routine

In [0]:
def compile_and_train(model, train_ds, configs, valid_ds=None):

  model.compile(
      loss=configs.LOSS,
      optimizer=configs.OPTIMIZER,
      metrics=configs.METRICS
  )

  history = model.fit(
      train_ds,
      validation_data = valid_ds,
      epochs=configs.EPOCHS,
      callbacks=configs.CALLBACKS
  )

  return model, history

#### Build and train the model

##### Params

In [0]:
train_configs = TrainConfigs()
train_configs.EPOCHS = 100
train_configs.LOSS = keras.losses.Huber()
train_configs.OPTIMIZER = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
train_configs.METRICS = ["mae"]
train_configs.CALLBACKS = [lr_schedule()]

hparams = HParams()
hparams.LAYERS_DIM = [100, 1]

In [0]:
model = build_model(hparams)

##### LR fine-tuning
Train a first model to search for the best LR ...

In [0]:
model, history = compile_and_train(
    model, train_set, train_configs
)

Check the __training loss__

In [0]:
plt.semilogx(history.history["lr"], history.history["loss"])
#plt.axis([1e-8, 1e-4, 0, 0.05])

##### Train the model

Train now the model for more epochs, on the "_fine-tuned_" LR

In [0]:
train_configs.EPOCHS = 500
train_configs.OPTIMIZER = keras.optimizers.SGD(lr=1e-6, momentum=0.9)
MODEL_NAME = "ann"
train_configs.CALLBACKS = [model_checkpoint(MODEL_NAME),
                           tensorboard_callback(MODEL_NAME),
                           early_stopping()]

Load TensorBoard ...

In [0]:
log_path = tb_log_path(MODEL_NAME)

In [0]:
%tensorboard --logdir logs

Train

In [0]:
model = build_model(hparams, False)
model, history = compile_and_train(
    model=model,
    train_ds=train_set,
    valid_ds=valid_set,
    configs=train_configs
)

Reload the best trained model

In [0]:
model = keras.models.load_model("ann.h5")

##### Check the predictions

In [0]:
split_time = int(.70*len(series))
ann_forecast = model.predict(series[np.newaxis, :, np.newaxis])
ann_forecast = ann_forecast[0, split_time - 1:-1, 0]

In [0]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid, ann_forecast, ["True", "Prediction"])

In [0]:
mae["ann"] = keras.metrics.mean_absolute_error(x_valid, ann_forecast).numpy()
mae["ann"]

### LSTM Model

##### Model definition

In [0]:
def build_model(hparams, log_summary=True):
  
  keras.backend.clear_session()
  tf.random.set_seed(42)
  np.random.seed(42)

  num_lstm_layers = len(hparams.LAYERS_DIM)

  model = keras.models.Sequential()
  model.add(
      keras.layers.LSTM(hparams.LAYERS_DIM[0], return_sequences=True, stateful=True,
                        batch_input_shape=[1, None, 1])
  )
  if num_lstm_layers > 1:
    for dim in hparams.LAYERS_DIM[1:]:
      model.add(keras.layers.LSTM(dim, return_sequences=True, stateful=True))

  model.add(keras.layers.Dense(1))
  model.add(keras.layers.Lambda(lambda x: x * 200.0))

  if log_summary:
    print("\n", model.summary(), "\n")

  return model

#### Training routine

In [0]:
def compile_and_train(model, train_ds, configs, valid_ds=None):

  model.compile(
      loss=configs.LOSS,
      optimizer=configs.OPTIMIZER,
      metrics=configs.METRICS
  )

  history = model.fit(
      train_ds,
      validation_data = valid_ds,
      epochs=configs.EPOCHS,
      callbacks=configs.CALLBACKS
  )

  return model, history

#### Build and train the model

##### Params

In [0]:
train_configs = TrainConfigs()
train_configs.EPOCHS = 100
train_configs.LOSS = keras.losses.Huber()
train_configs.OPTIMIZER = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
train_configs.METRICS = ["mae"]
train_configs.CALLBACKS = [lr_schedule(), ResetStatesCallback()]

hparams = HParams()
hparams.LAYERS_DIM = [100, 50]

##### LR fine-tuning
Train a first model to search for the best LR ...

In [0]:
model = build_model(hparams)

In [0]:
model, history = compile_and_train(
    model, train_set, train_configs
)

Check the __training loss__

In [0]:
plt.semilogx(history.history["lr"], history.history["loss"])
#plt.axis([1e-8, 1e-4, 0, 30])

##### Train the model

Train now the model for more epochs, on the "_fine-tuned_" LR

In [0]:
MODEL_NAME = "lstm"

train_configs.EPOCHS = 500
train_configs.OPTIMIZER = keras.optimizers.SGD(lr=1e-6, momentum=0.9)
train_configs.CALLBACKS = [ResetStatesCallback(),
                           model_checkpoint(MODEL_NAME),
                           early_stopping(),
                           tensorboard_callback(MODEL_NAME)]

Load TensorBoard ...

In [0]:
log_path = tb_log_path(MODEL_NAME)

In [0]:
%tensorboard --logdir logs

Train

In [0]:
model = build_model(hparams, False)
model, history = compile_and_train(
    model=model,
    train_ds=train_set,
    valid_ds=valid_set,
    configs=train_configs
)

Reload the best trained model

In [0]:
model = keras.models.load_model("lstm.h5")

#### Check the predictions

In [0]:
split_time = int(.70*len(series))
rnn_forecast = model.predict(series[np.newaxis, :, np.newaxis])
rnn_forecast = rnn_forecast[0, split_time - 1:-1, 0]

In [0]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid, rnn_forecast, ["True", "Prediction"])

In [0]:
mae["lstm"] = keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()
mae["lstm"]

### CNN Model

#### Model definition

In [0]:
def build_model(hparams, log_summary=True):
  
  keras.backend.clear_session()
  tf.random.set_seed(42)
  np.random.seed(42)

  num_layers = len(hparams.FILTERS)
  print(num_layers)

  model = keras.models.Sequential()
  model.add(
      keras.layers.Conv1D(filters=hparams.FILTERS[0],
                          kernel_size=hparams.KERNEL_SIZE[0],
                          strides=hparams.STRIDES,
                          padding="causal",
                          activation="relu",
                          input_shape=[None, 1])
  )
  if num_layers > 1:
    for dim in range(1, num_layers):
      model.add(keras.layers.Conv1D(filters=hparams.FILTERS[dim],
                                    kernel_size=hparams.KERNEL_SIZE[dim],
                                    strides=hparams.STRIDES,
                                    padding="causal",
                                    activation="relu"))

  model.add(keras.layers.Dense(1))
  model.add(keras.layers.Lambda(lambda x: x * 200.0))

  if log_summary:
    print("\n", model.summary(), "\n")

  return model

#### Training routine

#### Build and train the model

##### Params

In [0]:
train_configs = TrainConfigs()
train_configs.EPOCHS = 100
train_configs.LOSS = keras.losses.Huber()
train_configs.OPTIMIZER = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
train_configs.METRICS = ["mae"]
train_configs.CALLBACKS = [lr_schedule()]

hparams = HParams()
hparams.FILTERS = [32, 32]
hparams.KERNEL_SIZE = [5, 5]
hparams.STRIDES = 1

##### LR fine-tuning

In [0]:
model = build_model(hparams)

In [0]:
model, history = compile_and_train(
    model, train_set, train_configs
)

Check the __training loss__.

In [0]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 100])

##### Train the model

Train now the model for more epochs, on the "_fine-tuned_" LR

In [0]:
MODEL_NAME = "cnn"

train_configs.EPOCHS = 500
train_configs.OPTIMIZER = keras.optimizers.SGD(lr=3e-6, momentum=0.9)
train_configs.CALLBACKS = [model_checkpoint(MODEL_NAME),
                           early_stopping(),
                           tensorboard_callback(MODEL_NAME)]

Load TensorBoard ...

In [0]:
log_path = tb_log_path(MODEL_NAME)

In [0]:
%tensorboard --logdir logs

Train

In [0]:
model = build_model(hparams, False)
model, history = compile_and_train(
    model=model,
    train_ds=train_set,
    valid_ds=valid_set,
    configs=train_configs
)

Reload the best trained model

In [0]:
model = keras.models.load_model("cnn.h5")

##### Check the prediction

In [0]:
split_time = int(.70*len(series))
cnn_forecast = model.predict(series[np.newaxis, :, np.newaxis])
cnn_forecast = cnn_forecast[0, split_time - 1:-1, 0]

In [0]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid, cnn_forecast, ["True", "Prediction"])

In [0]:
mae["cnn"] = keras.metrics.mean_absolute_error(x_valid, cnn_forecast).numpy()
mae["cnn"]

### Comparison

In [0]:
comparison = pd.DataFrame(list(mae.items()), columns=["Model", "MAE"])
#comparison = comparison.set_index("Model")
comparison